# Metrics during GRACE era

In this notebook, we compare our TWS reconstruction to the GRACE, our target.

In [ ]:
import string

import cartopy.crs as ccrs
import cmcrameri.cm as cmc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import xarray as xr
from dask.diagnostics import ProgressBar
from matplotlib import ticker
from matplotlib_inline.backend_inline import set_matplotlib_formats

import deeprec  # noqa
from deeprec import metrics
from deeprec.regions import basins
from deeprec.utils import ROOT_DIR, repeat_by_weight

In [ ]:
# Register dask progress bar
ProgressBar(minimum=5).register()


In [ ]:
set_matplotlib_formats("retina")
plt.style.use(ROOT_DIR / "config/style_paper.mplstyle")
FIGURE_DIR = ROOT_DIR / "docs/figures/paper"


## Data Preprocessing

### Data Loading

Load and combine the different data sets (GRACE, ERA5, WGHM, and our and other reconstructions)

In [ ]:
our_store = (
    ROOT_DIR / "models/predictions/ensemble-mixed/ensemble-mixed_era_best-mae.zarr"
)
inputs_store = ROOT_DIR / "data/processed/inputs.zarr"
targets_store = ROOT_DIR / "data/processed/targets.zarr"
recs_store = ROOT_DIR / "data/processed/reconstructions.zarr"

tgts = xr.open_zarr(targets_store)
inps = xr.open_zarr(inputs_store)
recs = xr.open_zarr(recs_store)
our = xr.open_zarr(our_store).twsa.rename("twsa_our")


In [ ]:
# Merge all data sets
ds_all = xr.merge([tgts, inps, recs, our])

# Create and apply mask (land area and Greenland+Antarctica exclusion)
mask = (tgts.land_mask_jpl * tgts.land_mask_gsfc * inps.grl_ata_mask).compute()
ds_all = ds_all.where(mask == 1)

# Substract the GRACE baseline
baseline = slice("2004", "2009")
base_mean = ds_all.sel(time=baseline).mean("time")
ds_all = ds_all - base_mean


Specify variables of interest and their descriptive names:

In [ ]:
vars_verbose = {
    "twsa_our": "DeepRec (Ours)",
    "twsa_gap": "WGHM",
    "palazzoli_jpl_full": "Palazzoli's JPL Rec",
    "yin_csr_full": "Yin's CSR Rec",
    "li_csr_full": "Li's CSR Rec",
    "humphrey_gsfc_detrend": "Humphrey's GSFC Rec",
    "twsa_csr": "CSR (GRACE)",
}
target_name = "twsa_csr"
our_name = "twsa_our"

eval_names = list(vars_verbose.keys())
eval_names.remove(target_name)


## Error and correlation world maps

In [ ]:
ds_grid = ds_all[[target_name, *eval_names]]
# Only evaluate GRACE time
ds_grid = ds_grid.sel(time=slice("2002", "2023")).compute()

# Calculate basin averages
ds_basin: xr.Dataset = (
    # Select variables and time frame of interest
    ds_grid.chunk(time=10, lat=-1, lon=-1)
    .dr.select_basins(top=72)
    .dr.weight_lat()
    .mean(["lat", "lon"])
    .compute()
)


### Metrics Ours vs. GRACE (2002-2023)

In [ ]:
# Remove missing time steps - remove missing time steps in GRACE for all vars
ds_grid_our = ds_grid[[target_name, our_name]]
ds_grid_our = ds_grid_our.where(ds_grid_our.dr.time_notnull(), drop=True)

ds_basin_our = ds_basin[[target_name, our_name]]
ds_basin_our = ds_basin_our.where(ds_grid_our.dr.time_notnull(), drop=True)

times_our = ds_grid_our.get_index("time")
print(f"Period: {times_our[0].date()} to {times_our[-1].date()}")


In [ ]:
# RMSE
our_rmse_grid = metrics.rmse(
    ds_grid_our[target_name], ds_grid_our[our_name], dim="time", skipna=True
)
our_rmse_basin = metrics.rmse(
    ds_basin[target_name], ds_basin[our_name], dim="time", skipna=True
)

# NSE
our_nse_grid = metrics.nse(
    ds_grid_our[target_name], ds_grid_our[our_name], dim="time", skipna=True
)
our_nse_basin = metrics.nse(
    ds_basin[target_name], ds_basin[our_name], dim="time", skipna=True
)


### Metrics All vs. GRACE (2002-2019)

In [ ]:
# Remove missing time steps - remove missing time steps in GRACE for all vars
ds_grid_all = ds_grid[[target_name, *eval_names]]
ds_grid_all = ds_grid_all.where(ds_grid_all.dr.time_notnull(), drop=True)

ds_basin_all = ds_basin[[target_name, *eval_names]]
ds_basin_all = ds_basin_all.where(ds_grid_all.dr.time_notnull(), drop=True)

times_all = ds_grid_all.get_index("time")
print(f"Period: {times_all[0].date()} to {times_all[-1].date()}")


In [ ]:
# RMSE
da_rmse_grid = metrics.rmse(
    ds_grid[target_name], ds_grid[eval_names], dim="time", skipna=True
).to_dataarray("model")
da_rmse_basin = metrics.rmse(
    ds_basin[target_name], ds_basin[eval_names], dim="time", skipna=True
).to_dataarray("model")

# NSE
da_nse_grid = metrics.nse(
    ds_grid[target_name], ds_grid[eval_names], dim="time", skipna=True
).to_dataarray("model")
da_nse_basin = metrics.nse(
    ds_basin[target_name], ds_basin[eval_names], dim="time", skipna=True
).to_dataarray("model")


In [ ]:
rmse_kwargs = dict(
    cbar_kwargs=dict(
        location="right", aspect=15, shrink=0.93, pad=0.03, label="RMSE (mm)"
    ),
    cmap=cmc.batlow_r,
    coastlines=True,
    gridlines=True,
    coastlines_kwargs=dict(rasterized=True, linewidth=0.5),
    rasterized=True,
)
nse_kwargs = dict(
    cbar_kwargs=dict(location="right", aspect=15, shrink=0.93, pad=0.03, label="NSE"),
    cmap=cmc.navia_r,
    coastlines=True,
    gridlines=True,
    coastlines_kwargs=dict(rasterized=True, linewidth=0.5),
    rasterized=True,
)
basinlines_kwargs = dict(
    edgecolor="black",
    linewidth=0.4,
    facecolor="None",
    zorder=3.0,
    rasterized=True,
)
title_kwargs = dict(fontsize=16, weight="bold")


In [ ]:
# GeoDataFrame containing basin shapes and areas
gdf_basin_shapes = basins(top=72)

# Series of basin areas
s_basin_areas = gdf_basin_shapes.rename(
    columns={"riverbasin": "region", "sum_sub_ar": "area"}
).set_index("region")["area"]

# DataArray of basin areas, for weighting
da_basin_areas = xr.DataArray.from_series(s_basin_areas)

In [ ]:
print(f"{our_rmse_grid.min() = :.1f}")
print(f"{our_rmse_grid.max() = :.1f}\n")

print(f"{our_nse_grid.min() = :.1f}")
print(f"{our_nse_grid.max() = :.1f}\n")

print(f"{our_rmse_basin.min() = :.1f}")
print(f"{our_rmse_basin.max() = :.1f}\n")

print(f"{our_nse_basin.min() = :.1f}")
print(f"{our_nse_basin.max() = :.1f}\n")


In [ ]:
print(f"{our_rmse_grid.dr.weight_lat().mean() = :.1f}")
print(f"{our_rmse_grid.dr.weight_lat().quantile(0.9) = :.1f}\n")

print(f"{our_nse_grid.dr.weight_lat().mean() = :.1f}")
print(f"{our_nse_grid.dr.weight_lat().quantile(0.1) = :.1f}\n")

print(f"{our_rmse_basin.weighted(da_basin_areas).mean() = :.1f}")
print(f"{our_rmse_basin.weighted(da_basin_areas).quantile(0.9) = :.1f}\n")

print(f"{our_nse_basin.weighted(da_basin_areas).mean() = :.1f}")
print(f"{our_nse_basin.weighted(da_basin_areas).quantile(0.1) = :.1f}\n")


In [ ]:
fig, axs = plt.subplots(
    2, 2, subplot_kw={"projection": ccrs.EqualEarth()}, figsize=(7.2, 3.5)
)

# Grid scale plots
our_rmse_grid.dr.projplot(vmin=0, vmax=100, ax=axs[0][0], **rmse_kwargs)
our_nse_grid.dr.projplot(vmin=0.0, vmax=1.0, ax=axs[1][0], **nse_kwargs)

# Basin scale plots
our_rmse_basin.dr.projplot_basins(
    our_rmse_grid,
    vmin=0,
    vmax=50,
    ax=axs[0][1],
    **rmse_kwargs,
)
our_nse_basin.dr.projplot_basins(
    our_nse_grid,
    vmin=0.7,
    vmax=1.0,
    ax=axs[1][1],
    **nse_kwargs,
)

# Add basinlines
for ax in [axs[0][1], axs[1][1]]:
    gdf_basin_shapes.dr.projplot(ax=ax, **basinlines_kwargs)

# Titles
axs[0][0].set(title="Grid scale")
axs[0][1].set(title="Basin scale")


## Boxplot

In [ ]:
# Convert DataArray to DataFrame
df_nse = da_nse_basin.to_dataframe(name="nse").reset_index()
df_nse["model_verbose"] = df_nse["model"].map(vars_verbose)

# Convert area to integer (we define the smallest area as 10)
s_basin_ints = (s_basin_areas / s_basin_areas.min() * 10).round().astype(int)

# Merge with area integers
df_nse_area = df_nse.merge(s_basin_ints, left_on="region", right_index=True)

# Create weighted dataframe (columns repeated according to area integers)
df_nse_weighted = repeat_by_weight(df_nse_area, "area")

In [ ]:
# Order by increasing mean
my_order = (
    df_nse.groupby(by=["model_verbose"])["nse"]
    .mean()
    # Sort GRACE to the beginning
    .sort_values(na_position="first")
    .index
)
my_order


In [ ]:
colors = sns.color_palette("tab20")
colors

In [ ]:
# Create dictionaries for color lookup
dark_colors_dict = {
    "twsa_our": colors[0],
    "twsa_csr": colors[2],
    "twsa_gap": colors[4],
    "yin_csr_full": colors[6],
    "palazzoli_jpl_full": colors[8],
    "li_csr_full": colors[10],
    "humphrey_gsfc_detrend": colors[12],
    "twsa_gmsl_alt": colors[16],
    "twsa_gmsl_obs": colors[18],
}
light_colors_dict = {
    "twsa_our": colors[1],
    "twsa_csr": colors[3],
    "twsa_gap": colors[5],
    "palazzoli_jpl_full": colors[7],
    "yin_csr_full": colors[9],
    "li_csr_full": colors[11],
    "humphrey_gsfc_detrend": colors[13],
}


In [ ]:
fig, ax = plt.subplots(figsize=(7.2, 2.2))
sns.boxplot(
    df_nse_weighted,
    x="nse",
    y="model_verbose",
    hue="model",
    width=0.5,
    ax=ax,
    order=my_order,
    palette=dark_colors_dict,
    showfliers=False,
    saturation=1,
    legend=False,
)
ax.set(xlabel="NSE", ylabel=None, xlim=(0, 1))
ax.xaxis.grid()


## Bar chart

Create a bar chart which shows grid scale and basin scale NSE in one chart

In [ ]:
da_rmse_grid_avg = (
    da_rmse_grid.dr.weight_lat().mean(["lat", "lon"]).rename("Grid scale RMSE")
)
da_rmse_basin_avg = (
    da_rmse_basin.weighted(da_basin_areas).mean("region").rename("Basin scale RMSE")
)

# Concatenate along new "scale " dimension
da_rmse_avg = xr.concat(
    [da_rmse_grid_avg, da_rmse_basin_avg],
    pd.Index(["Grid scale", "Basin scale"], name="scale"),
)

In [ ]:
df_rmse_avg = (
    da_rmse_avg.to_pandas()
    .rename(columns=vars_verbose)
    .reset_index()
    .melt(id_vars="scale", value_name="rmse")
)
df_rmse_avg

In [ ]:
my_order = (
    df_rmse_avg.query("scale == 'Basin scale'")
    .sort_values("rmse", ascending=False)
    .model
)
my_order


In [ ]:
fig, ax = plt.subplots(figsize=(7.2, 3))
sns.barplot(
    df_rmse_avg,
    x="rmse",
    y="model",
    hue="scale",
    width=0.6,
    gap=0.1,
    ax=ax,
    palette="Paired",
    order=my_order,
)
ax.set(xlabel="RMSE (mm)", ylabel=None, xlim=(0, 100))
ax.legend(loc="lower right")
ax.xaxis.grid()


## Scatter plots of signal components

### Perform linear trend / multi-seasonal decomposition

We use the following function to approximate the constant $a$, the linear trend $b$, and the annual and semi-annual sinusoidal curves:

$$
y(t) = a + b \, t + c \, \sin(2π  \, t) + d \, \cos(2π \, t) + e \, \sin(4π \, t) + f \cos(4π \, t) + r
$$

In [ ]:
YEAR_NS = 365.25 * 24 * 60 * 60 * 1e9


def decomp_func(
    t: np.ndarray, a: float, b: float, c: float, d: float, e: float, f: float
) -> np.ndarray:
    # Convert t from nanosecons to years
    t = t / YEAR_NS

    linear = a + b * t
    annual = c * np.sin(2 * np.pi * t) + d * np.cos(2 * np.pi * t)
    semian = e * np.sin(4 * np.pi * t) + f * np.cos(4 * np.pi * t)

    return linear + annual + semian


In [ ]:
# Apply decomposition
# Curvefit skips missing values per default
fit = ds_basin_our.to_dataarray("model").curvefit("time", decomp_func)
fit


### Calculate slope and amplitudes

In [ ]:
# Linear trend: mm / yr
da_lintrend = fit.curvefit_coefficients.sel(param="b")

da_lintrend.attrs = {"long_name": "Trend", "unit": "mm / yr"}


Calculate annual amplitude:

$$
A = \sqrt{c^2 + d^2}
$$

In [ ]:
c = fit.curvefit_coefficients.sel(param="c")
d = fit.curvefit_coefficients.sel(param="d")
e = fit.curvefit_coefficients.sel(param="e")
f = fit.curvefit_coefficients.sel(param="f")

da_amp_annual = np.sqrt(c**2 + d**2)
da_amp_semian = np.sqrt(e**2 + f**2)

da_amp_annual.attrs = {"long_name": "Annual amplitude", "unit": "mm"}
da_amp_semian.attrs = {"long_name": "Semi-annual amplitude", "unit": "mm"}


In [ ]:
# Place components in dictionaries for easier batch handling
da_comps = {
    "lintrend": da_lintrend,
    "amp_annual": da_amp_annual,
    "amp_semian": da_amp_semian,
}
df_comps = {}
pcc_scores = {}

for name, da in da_comps.items():
    # Convert DataArrays to DataFrames for plotting with Seaborn
    df = (
        da.to_pandas()
        .transpose()
        .reset_index()
        # Add basin size column (in Million km)
        .merge(s_basin_areas / 1e6, on="region")
        # Sort after area so large basins are printed lastly
        .sort_values("area")
    )
    df_comps[name] = df

    # Calculate PCC
    pcc_scores[name] = metrics.pearson_r(
        da.sel(model=target_name), da.sel(model=our_name), weights=da_basin_areas
    )


In [ ]:
# Units
units = {
    "lintrend": "mm yr$^-1$",
    "amp_annual": "mm",
    "amp_semian": "mm",
}
# Verbose names
names_verbose = {
    "lintrend": "Linear\ntrend",
    "amp_annual": "Annual\namplitude",
    "amp_semian": "Semiannual\namplitude",
}


In [ ]:
print(f"{da_lintrend.min() = :.2f}")
print(f"{da_lintrend.max() = :.2f}")

print(f"{da_amp_annual.min() = :.2f}")
print(f"{da_amp_annual.max() = :.2f}")

print(f"{da_amp_semian.min() = :.2f}")
print(f"{da_amp_semian.max() = :.2f}")


In [ ]:
# Limits
lims = {
    "lintrend": (-20, 10),
    "amp_annual": (0, 250),
    "amp_semian": (0, 80),
}
# Major locators
locs = {
    "lintrend": ticker.MultipleLocator(10),
    "amp_annual": ticker.MultipleLocator(50),
    "amp_semian": ticker.MultipleLocator(20),
}

In [ ]:
fig, axs = plt.subplots(ncols=3, figsize=(7.2, 2.2))

for i, (ax, (name, df)) in enumerate(zip(axs, df_comps.items())):
    sns.scatterplot(
        df,
        x=target_name,
        y=our_name,
        ax=ax,
        size="area",
        legend=True if i == 0 else False,
    )

    # Unpack dicts
    name_verbose = names_verbose[name]
    unit = units[name]
    lim = lims[name]
    loc = locs[name]
    pcc = pcc_scores[name]

    ax.set(
        title=name_verbose,
        xlabel=f"CSR ({unit})",
        ylabel=f"DeepRec ({unit})",
        xlim=lim,
        ylim=lim,
        aspect="equal",
    )
    ax.xaxis.set_major_locator(loc)
    ax.yaxis.set_major_locator(loc)
    ax.xaxis.grid(True)
    ax.yaxis.grid(True)

    # Add diagonal line
    ax.plot(
        lim,
        lim,
        color="gray",
        linestyle="dashed",
        linewidth=0.8,
        zorder=0.0,
    )

    # Add score
    ax.text(1.0, 0.07, f"PCC = {pcc:.2f}", ha="right", transform=ax.transAxes)


# Place legend outside of axes
axs[0].legend().remove()
legend = fig.legend(loc="outside right", title="Area\n($10^6$ km$^2$)")
legend.get_title().set_ha("center")


## Combine Everything

In [ ]:
fig = plt.figure(figsize=(7.2, 8.0))
subfigs = fig.subfigures(nrows=3, hspace=0.05, height_ratios=[1.5, 1, 1.2])

axs_geo = subfigs[0].subplots(2, 2, subplot_kw={"projection": ccrs.EqualEarth()})
axs_scatter = subfigs[1].subplots(1, 3)
ax_bar = subfigs[2].subplots(1, 1)

axs_rmse, axs_nse = axs_geo

### MAP PLOTS ###

# Grid scale plots
our_rmse_grid.dr.projplot(vmin=0, vmax=100, ax=axs_rmse[0], **rmse_kwargs)
our_nse_grid.dr.projplot(vmin=0.0, vmax=1.0, ax=axs_nse[0], **nse_kwargs)

# Basin scale plots
our_rmse_basin.dr.projplot_basins(
    our_rmse_grid,
    vmin=0,
    vmax=50,
    ax=axs_rmse[1],
    **rmse_kwargs,
)
our_nse_basin.dr.projplot_basins(
    our_nse_grid,
    vmin=0.7,
    vmax=1.0,
    ax=axs_nse[1],
    **nse_kwargs,
)

# Add basinlines
for ax in [axs_rmse[1], axs_nse[1]]:
    gdf_basin_shapes.dr.projplot(ax=ax, **basinlines_kwargs)

# Titles
time_str_our = f"{times_our[0].year}--{times_our[-1].year}"
axs_rmse[0].set(title="Grid scale, " + time_str_our)
axs_rmse[1].set(title="Basin scale, " + time_str_our)

### SCATTER PLOT ###

for i, (ax, (name, df)) in enumerate(zip(axs_scatter, df_comps.items())):
    sns.scatterplot(
        df,
        x=target_name,
        y=our_name,
        ax=ax,
        size="area",
        legend=True if i == 0 else False,
    )

    # Unpack dicts
    name_verbose = names_verbose[name]
    unit = units[name]
    lim = lims[name]
    loc = locs[name]
    pcc = pcc_scores[name]

    ax.set(
        title=name_verbose,
        xlabel=f"CSR ({unit})",
        ylabel=f"DeepRec ({unit})",
        xlim=lim,
        ylim=lim,
        aspect="equal",
    )
    ax.xaxis.set_major_locator(loc)
    ax.yaxis.set_major_locator(loc)
    ax.xaxis.grid(True)
    ax.yaxis.grid(True)

    # Add diagonal line
    ax.plot(
        lim,
        lim,
        color="gray",
        linestyle="dashed",
        linewidth=0.8,
        zorder=0,
    )

    # Add PCC score
    ax.text(1.0, 0.07, f"PCC = {pcc:.2f}", ha="right", transform=ax.transAxes)


# Place legend outside of axes
axs_scatter[0].legend().remove()
legend = subfigs[1].legend(loc="outside right", title="Area\n($10^6$ km$^2$)")
legend.get_title().set_ha("center")

### BAR CHART ###

sns.barplot(
    df_rmse_avg,
    x="rmse",
    y="model",
    hue="scale",
    width=0.6,
    gap=0.1,
    ax=ax_bar,
    palette="Paired",
    order=my_order,
)
time_str_all = f"{times_all[0].year}--{times_all[-1].year}"
ax_bar.set(
    xlabel="RMSE (mm)",
    ylabel=None,
    xlim=(0, 100),
    title="Global average error, " + time_str_all,
)
ax_bar.xaxis.grid()
# Place legend outside of axis
ax_bar.legend().remove()
subfigs[2].legend(loc="outside right")

### LETTERS ###

for n, ax in enumerate(axs_geo.flat):
    ax.text(
        0.0,
        1.0 - 0.08,
        string.ascii_lowercase[n],
        transform=ax.transAxes,
        size="x-large",
        weight="bold",
    )
for n, ax in enumerate(axs_scatter):
    ax.text(
        0.0,
        1.0 + 0.15,
        string.ascii_lowercase[len(axs_geo.flat) + n],
        ha="center",
        transform=ax.transAxes,
        size="x-large",
        weight="bold",
    )
ax_bar.text(
    0.0,
    1.0 + 0.06,
    string.ascii_lowercase[len(axs_geo.flat) + len(axs_scatter.flat)],
    ha="center",
    transform=ax_bar.transAxes,
    size="x-large",
    weight="bold",
)

fig.savefig(FIGURE_DIR / "grace_comp_r.pdf", backend="pgf")
